In [6]:
library(limma) 
library(edgeR)


In [2]:
# load data
counts = read.csv('../../bulkRNAseq/counts.txt', comment="#", sep="\t", check.names=FALSE)
metadata = read.csv('../../bulkRNAseq/abca7_rna_seq_metadata_with_seqIDs.csv', check.names=FALSE)
metadata$SeqID = gsub(" ", "", metadata$SeqID)

sam_columns = grep("\\.sam$", names(counts), value = TRUE)
counts_subset = counts[,sam_columns[2:length(sam_columns)]]
colnames(counts_subset) <- sapply(strsplit(colnames(counts_subset), "_"), function(x) x[2])
names(counts_subset) == metadata$SeqID

counts_pc = counts_subset[counts$gene_type=='protein_coding',]
names = counts$Geneid[counts$gene_type=='protein_coding']
gene_names = counts$gene_name[counts$gene_type=='protein_coding']
row.names(counts_pc) = names

lib.size = colSums(counts_subset)


[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [3]:
# create DGEList
dge_obj = DGEList(counts=counts_pc,genes=gene_names, samples=metadata, lib.size=lib.size)
isexpr = rowSums(cpm(dge_obj) >= 1) >= 1
dge_obj = dge_obj[isexpr,]

# compute choline-assoc genes
subset_x = dge_obj[,grepl("batch 1", dge_obj$samples$Notes)]
subset_x = subset_x[,subset_x$samples$Line=='Y622']
mod1 = model.matrix(~subset_x$samples$Treatment=='Choline2Weeks')
colnames(mod1) = c('intercept', 'condition')
v = voom(subset_x, design=mod1, plot = FALSE)
fit = lmFit(v, design=mod1)
fit = eBayes(fit)
res_choline = topTable(fit, coef='condition', n=Inf, sort.by="P")

# compute choline-assoc genes
subset_x = dge_obj[,grepl("batch 2", dge_obj$samples$Notes)]
subset_x = subset_x[,subset_x$samples$Line=='Y622']
mod1 = model.matrix(~subset_x$samples$Treatment=='Choline2Weeks')
colnames(mod1) = c('intercept', 'condition')
v = voom(subset_x, design=mod1, plot = FALSE)
fit = lmFit(v, design=mod1)
fit = eBayes(fit)
res_choline_batch2 = topTable(fit, coef='condition', n=Inf, sort.by="P")

# compute y622-assoc genes
subset_x = dge_obj[,grepl("batch 1", dge_obj$samples$Notes)]
subset_x = subset_x[,subset_x$samples$Treatment=='H20' & subset_x$samples$Line!='G2']
mod1 = model.matrix(~subset_x$samples$Line=='Y622')
colnames(mod1) = c('intercept', 'condition')
v = voom(subset_x, design=mod1, plot = FALSE)
fit = lmFit(v, design=mod1)
fit = eBayes(fit)
res_y622 = topTable(fit, coef='condition', n=Inf, sort.by="P")


# compute y622-assoc genes
subset_x = dge_obj[,grepl("batch 1", dge_obj$samples$Notes)]
subset_x = subset_x[,subset_x$samples$Treatment=='H20' & subset_x$samples$Line!='Y622']
mod1 = model.matrix(~subset_x$samples$Line=='G2')
colnames(mod1) = c('intercept', 'condition')
v = voom(subset_x, design=mod1, plot = FALSE)
fit = lmFit(v, design=mod1)
fit = eBayes(fit)
res_g2 = topTable(fit, coef='condition', n=Inf, sort.by="P")


In [14]:
table(metadata$Notes, metadata$Treatment, metadata$Line)

, ,  = Control

                                                                        
                                                                         Choline2Weeks
  diff & extraction batch 1 (1 well, DOI:101624, JennyExtraction 111324)             0
  diff & extraction batch 2 (2 wells pooled)                                         0
                                                                        
                                                                         H20
  diff & extraction batch 1 (1 well, DOI:101624, JennyExtraction 111324)   2
  diff & extraction batch 2 (2 wells pooled)                               0

, ,  = G2

                                                                        
                                                                         Choline2Weeks
  diff & extraction batch 1 (1 well, DOI:101624, JennyExtraction 111324)             0
  diff & extraction batch 2 (2 wells pooled)                                         0


In [4]:
write.csv(res_y622, "../../bulkRNAseq/y622_degs.csv")
write.csv(res_choline, "../../bulkRNAseq/choline_degs.csv")
write.csv(res_g2, "../../bulkRNAseq/g2_degs.csv")
write.csv(res_choline_batch2, "../../bulkRNAseq/choline_batch2_degs.csv")